In [1]:
import pandas as pd
import numpy as np
from pandas import ExcelWriter
import xlsxwriter
import warnings
warnings.simplefilter('ignore')

In [ ]:
curr_report_path = input('This quarter report: ') # 
prev_report_path = input('Last quarter report: ') # 

In [3]:
curr_report = pd.read_excel(curr_report_path, sheet_name=None)
prev_report = pd.read_excel(prev_report_path, sheet_name=None)

In [4]:
sheet_list = list(curr_report.keys())

df_dict = {}

for sheet in sheet_list:
    
    curr_df = curr_report.get(sheet)
    prev_df = prev_report.get(sheet)
    
    # create column for identifying key
    curr_df['key'] = curr_df['Party ID'] + curr_df['Inconsistency']
    prev_df['key'] = prev_df['Party ID'] + prev_df['Inconsistency']
    
    # identify prev_report records that are not resolved 
    curr_list = curr_df['key'].tolist()
    prev_list = prev_df['key'].tolist()
    
    # keep prev_report records that are still pending
    prev_df['exist in current'] = prev_df['key'].isin(curr_list)
    
    # remove curr_report records that are already present in prev_reports
    curr_df = curr_df[~curr_df['key'].isin(prev_list)]
    
    result_df = pd.concat([prev_df[prev_df['exist in current']==True], curr_df])
    
    df_dict[sheet] = result_df

In [5]:
fin_sheet_list = list(df_dict.keys())

writer = ExcelWriter("NP_2023_Q1_2_3.xlsx", mode='w', engine='xlsxwriter')

for sheet in fin_sheet_list:
    result = df_dict.get(sheet)
    result.to_excel(writer, sheet, index=False)
    
    workbook = writer.book
    format = workbook.add_format({'bg_color': '#FDE9D9'}) # use #FDE9D9
    worksheet = writer.sheets[sheet]
    
    # select columns to colour
    # note: at time of running, test reports only have "status of cleanup" and "completion date" as common columns
    cols = ['Remarks','Action Required','Action Team','Status of Cleanup','Completion Date','Valid Exception']
    
    # apply colour to each column
    for col in cols:
        col_num = result.columns.get_loc(col)
        worksheet.set_column(col_num, col_num, 20, format)
    

writer.save()

In [6]:
print('Completed.')

Completed.
